In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [7]:
import time
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

 
data = pd.read_csv('../input/titanic/train.csv')
data = data.drop(['PassengerId' , 'Name' , 'Ticket' ,'Cabin' ] , axis = 'columns')

def get_score(model , X_train , X_test, Y_train , Y_test):
	model.fit(X_train ,Y_train)
	return model.score(X_test ,Y_test)

def fill_missing(data):
    for col in data.columns:
        data[col] = data[col].fillna(data[col].mean())
 
        
le =LabelEncoder()
label =le.fit_transform(data['Sex'])
data =data.drop(['Sex'] , axis = 'columns')
data['Sex'] = label

replacer =[]

for i in data['Embarked']:
    if i=='S':
        replacer.append(1)
    else:
        replacer.append(0)

data['Embarked'].replace('S',1)
data =data.drop(['Embarked'] , axis = 'columns')
data['Embarked'] = replacer

fill_missing(data)


X = data.drop(['Survived'] , axis = 'columns')
Y = data['Survived']
X.head()
 

 


,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked
0,3,22.0,1,0,7.2500,1,1
1,1,38.0,1,0,71.2833,0,0
2,3,26.0,0,0,7.9250,0,1
3,1,35.0,1,0,53.1000,0,1
4,3,35.0,0,0,8.0500,1,1


In [8]:
X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size=0.20, random_state=0)
#classification models
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier , GradientBoostingClassifier


In [56]:
#adaboosting


model =DecisionTreeClassifier(criterion = 'entropy',  max_depth =1)
adaboost = AdaBoostClassifier(base_estimator=model  , n_estimators=100 ,learning_rate=1)
ada_model = adaboost.fit(X_train ,Y_train)
Y_pred = ada_model .predict(X_test)
print('for decision tree model = ' , get_score(model ,X_train, X_test, Y_train, Y_test))
print('for the adaboosting model = ' , get_score(ada_model ,X_train, X_test, Y_train, Y_test))


for decision tree model =  0.7877094972067039
for the adaboosting model =  0.8100558659217877


In [57]:
#gradientboosting


model =DecisionTreeClassifier(criterion = 'entropy',  max_depth =1)
gradientboost = GradientBoostingClassifier(  n_estimators=100 ,learning_rate=1 , max_depth=1 ,random_state=0)

gradient_model = gradientboost.fit(X_train ,Y_train)
Y_pred = gradient_model .predict(X_test)
print('for decision tree model = ' , get_score(model ,X_train, X_test, Y_train, Y_test))
print('for the adaboosting model = ' , get_score(gradient_model ,X_train, X_test, Y_train, Y_test))


for decision tree model =  0.7877094972067039
for the adaboosting model =  0.8212290502793296


In [66]:
#parameter tuning
from sklearn.model_selection import GridSearchCV
param_grid   = {
    'learning_rate':[.1,.2,1,2] ,
    'n_estimators':[100,500,250] , 
    'max_depth':[2],
    'random_state':[0]
}


grid_search_cv   = GridSearchCV(param_grid=param_grid ,estimator =  GradientBoostingClassifier() , scoring='accuracy')
grid_search_cv.fit(X_train , Y_train)
print('for decision tree model = ' , get_score(model ,X_train, X_test, Y_train, Y_test))
print('for the tuned model = ' , get_score(grid_search_cv ,X_train, X_test, Y_train, Y_test))


for decision tree model =  0.7877094972067039
for the tuned model =  0.2924901185770752


In [9]:
print(grid_search_cv.best_params_)
print(grid_search_cv.best_score_)


NameError: name 'grid_search_cv' is not defined

In [24]:
#xgboost
from xgboost import XGBClassifier

start = time.time()

xgboost = XGBClassifier(eta=1.0 ,gamma=10)
xgboost.fit(X_train , Y_train)
print('for decision tree model = ' , get_score(model ,X_train, X_test, Y_train, Y_test) ,'time = ',time.time()-start)
start = time.time()
print('score for xgbclassfifier = '  ,get_score(xgboost,X_train, X_test, Y_train, Y_test ) , 'time = ' ,time.time()-start)




for decision tree model =  0.8212290502793296 time =  0.13348746299743652
score for xgbclassfifier =  0.8268156424581006 time =  0.0725712776184082


In [33]:
#paramater tuning xgboosting
from sklearn.model_selection import GridSearchCV

param_grid   = {
    'eta':[.1,.2,1,2,3] ,
    'gamma':[1,10,100,500] 
}


grid_search_cv   = GridSearchCV(param_grid=param_grid ,estimator =  XGBClassifier() , scoring='accuracy')
grid_search_cv.fit(X_train , Y_train)
print('for decision tree model = ' , get_score(model ,X_train, X_test, Y_train, Y_test))
print('for the tuned xgboost model = ' , get_score(grid_search_cv ,X_train, X_test, Y_train, Y_test))





for decision tree model =  0.8212290502793296
for the tuned xgboost model =  0.8491620111731844


In [34]:
print(grid_search_cv.best_params_)
print(grid_search_cv.best_score_)


{'eta': 0.1, 'gamma': 1}
0.8356446370530879
